# Sesión 1 · Introducción a ODEs y Circuitos Biológicos

Este cuaderno está diseñado para el **profesor**. Incluye:
- Código modular para simular circuitos biológicos simples.
- Ejercicios basados en el Capítulo 1 del libro de Uri Alon.
- Soluciones comentadas y notas pedagógicas.

**Objetivo:** Mostrar cómo los circuitos biológicos procesan información mediante ecuaciones diferenciales ordinarias (ODEs) y cómo sus propiedades dependen de parámetros clave.

**Lecturas recomendadas:**
- Dennis Bray, *Wetware* (Capítulos 3 y 4)
- Uri Alon, *An Introduction to Systems Biology* (Capítulo 1)


In [ ]:
# Módulos básicos para simulación
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import pandas as pd

# Módulo 1: Modelo ODE

def model_gene_activation(t, y, params):
    S_t = params["inputs"]["S"](t)
    Pmax = params["model"].get("Pmax", 1.0)
    K_act = params["model"].get("K", 0.5)
    n_act = params["model"].get("n", 2)
    alpha = params["model"].get("alpha", 0.1)
    hill_activation = params["functions"]["hill_activation"]
    prod = Pmax * hill_activation(S_t, K_act, n_act)
    dy = prod - alpha * y[0]
    return [dy]

# Módulo 2: Configuración

def hill_activation(x, K, n):
    return (x**n) / (K**n + x**n)

def step_input(t, t_on=10.0, val_before=0.0, val_after=1.0):
    return val_before if t < t_on else val_after

def pulse_input(t, t_on=10.0, t_off=30.0, val=1.0):
    return val if t_on <= t < t_off else 0.0

def build_config(model_params=None, input_func=None, functions=None, y0=None, t_span=(0,60), t_eval=None):
    if model_params is None:
        model_params = {"Pmax":1.0,"K":0.5,"n":2,"alpha":0.1}
    if input_func is None:
        input_func = lambda t: step_input(t)
    if functions is None:
        functions = {"hill_activation":hill_activation}
    if y0 is None:
        y0 = [0.0]
    if t_eval is None:
        t_eval = np.linspace(t_span[0], t_span[1], 900)
    params = {"model":model_params,"inputs":{"S":input_func},"functions":functions,"meta":{"name":"Circuito simple"}}
    return params,y0,t_span,t_eval

# Módulo 3: Ejecución

def run_simulation(model_func, params, y0, t_span, t_eval):
    sol = solve_ivp(lambda t,y:model_func(t,y,params), t_span=t_span, y0=y0, t_eval=t_eval, method="RK45", rtol=1e-6, atol=1e-9)
    return sol

# Módulo 4: Visualización

def plot_timeseries(sol, labels=None, title=None):
    plt.figure(figsize=(7,4))
    for i in range(sol.y.shape[0]):
        lab = labels[i] if labels and i < len(labels) else f"var{i}"
        plt.plot(sol.t, sol.y[i], label=lab)
    plt.xlabel("Tiempo")
    plt.ylabel("Concentración")
    plt.title(title or "Dinámica temporal")
    plt.legend()
    plt.show()


In [ ]:
# Funciones de análisis

def response_time(sol, frac=0.5):
    steady = sol.y[0][-1]
    target = frac * steady
    idx = np.where(sol.y[0] >= target)[0][0]
    return sol.t[idx]

def steady_state(sol):
    return sol.y[0][-1]


## Ejercicios sobre regulación simple

### Ejercicio 1: Tiempo de respuesta
Simula el circuito con diferentes valores de alpha y calcula t_1/2.

### Ejercicio 2: Sensibilidad y cooperatividad
Varía n y observa la forma de la curva.

### Ejercicio 3: Umbral de activación
Modifica K y analiza la sensibilidad.

### Ejercicio 4: Ganancia y saturación
Cambia la amplitud de la entrada.

### Ejercicio 5: Robustez ante ruido
Introduce noisy_input y compara variabilidad.


In [ ]:
# Solución Ejercicio 1
for alpha in [0.05,0.1,0.2]:
    params,y0,t_span,t_eval = build_config(model_params={"Pmax":1.0,"K":0.5,"n":2,"alpha":alpha}, input_func=lambda t: step_input(t,t_on=10))
    sol = run_simulation(model_gene_activation, params, y0, t_span, t_eval)
    t_half = response_time(sol)
    print(f"alpha={alpha}, t_1/2={t_half:.2f}")
    plot_timeseries(sol, labels=["Y"], title=f"alpha={alpha}")
